In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
from afl.data import afltables

df = afltables.get_fixtures()

def sigmoid(x, width=100):
    return 1.0 / (1 + np.exp(-x/width))

df['home_squashed_margin'] = df['home_margin'].pipe(sigmoid)  # a width of 100 returns a similiar looking distribution to the home_points_ratio
# its possible we want to use the same width as the expected_home_squashed_margin, so that the ratings diff predicts the score margin?
df.tail()

,date,round,home_team,home_score,away_team,away_score,venue,season,home_points,away_points,home_points_ratio,home_margin,winner,home_win_draw_loss,round_type,round_number,home_state,away_state,is_interstate,home_squashed_margin
matchid,,,,,,,,,,,,,,,,,,,,
15448,2019-04-20,R5,Melbourne,7.13.55,St Kilda,15.5.95,M.C.G.,2019,55,95,0.366667,-40,away,0.0,regular,5,VIC,VIC,False,0.401312
15449,2019-04-20,R5,Richmond,13.11.89,Sydney,10.7.67,Docklands,2019,89,67,0.570513,22,home,1.0,regular,5,VIC,NSW,True,0.554779
15450,2019-04-21,R5,Footscray,7.15.57,Carlton,15.11.101,Docklands,2019,57,101,0.360759,-44,away,0.0,regular,5,VIC,VIC,False,0.391741
15451,2019-04-21,R5,Adelaide,18.11.119,Gold Coast,6.10.46,Adelaide Oval,2019,119,46,0.721212,73,home,1.0,regular,5,SA,QLD,True,0.674805
15452,2019-04-22,R5,Hawthorn,13.12.90,Geelong,17.11.113,M.C.G.,2019,90,113,0.443350,-23,away,0.0,regular,5,VIC,GEE,True,0.442752


In [3]:
d = pd.DataFrame({'matchid': [1, 1, 2, 2], 'team': ['a', 'b', 'b', 'a'], 'points': [100, 90, 80, 90], 'is_home': [True, False, True, False]})
d = d.set_index(['matchid', 'team', 'is_home'])

In [6]:
def longify(df, meta_cols=('season', 'round', 'matchid', 'date', 'venue')):
    meta_cols = list(meta_cols)
    
    def _gen(df):
        for rec in df.to_dict('records'):
            meta = {k: rec[k] for k in meta_cols}
            yield {'team': rec['home_team'], 'points': rec['home_points'], 'win_draw_loss': rec['home_win_draw_loss'], **meta}
            yield {'team': rec['away_team'], 'points': rec['away_points'], 'win_draw_loss': 1-rec['home_win_draw_loss'], **meta}

    return pd.DataFrame(_gen(df)).set_index(meta_cols)

In [12]:
# longify(df.reset_index()).tail(8)